<a href="https://colab.research.google.com/github/liao1230ho123/ssneural/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import random
import numpy as np

In [0]:
BASE = 10

#PRECISION_INTEGRAL = 1
#PRECISION_FRACTIONAL = 6
#Q = 10000019

PRECISION_INTEGRAL = 8
PRECISION_FRACTIONAL = 8
Q = 293973345475167247070445277780365744413

PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL

assert(Q > BASE**PRECISION)

In [0]:
def encode(rational):
    upscaled = int(rational * BASE**PRECISION_FRACTIONAL)
    field_element = upscaled % Q
    return field_element

def decode(field_element):
    upscaled = field_element if field_element <= Q/2 else field_element - Q
    rational = upscaled / BASE**PRECISION_FRACTIONAL
    return rational

In [0]:
def share(secret):
    first  = random.randrange(Q)
    second = random.randrange(Q)
    third  = (secret - first - second) % Q
    return [first, second, third]

def reconstruct(sharing):
    return sum(sharing) % Q

def reshare(x):
    Y = [ share(x[0]), share(x[1]), share(x[2]) ]
    return [ sum(row) % Q for row in zip(*Y) ]

In [0]:
def add(x, y):
    return [ (xi + yi) % Q for xi, yi in zip(x, y) ]

def sub(x, y):
    return [ (xi - yi) % Q for xi, yi in zip(x, y) ]
    
def imul(x, k):
    return [ (xi * k) % Q for xi in x ]

In [0]:
INVERSE = 104491423396290281423421247963055991507 # inverse of BASE**FRACTIONAL_PRECISION
KAPPA = 6 # leave room for five digits overflow before leakage

assert((INVERSE * BASE**PRECISION_FRACTIONAL) % Q == 1)
assert(Q > BASE**(2*PRECISION + KAPPA))
def truncate(a):
    # map to the positive range
    b = add(a, share(10**(6+6-1)))
    # apply mask known only by P0, and reconstruct masked b to P1 or P2
    mask = random.randrange(Q) % 10**(6+6+KAPPA)
    mask_low = mask % 10**6
    b_masked = reconstruct(add(b, share(mask)))
    # extract lower digits
    b_masked_low = b_masked % 10**6
    b_low = sub(share(b_masked_low), share(mask_low))
    # remove lower digits
    c = sub(a, b_low)
    # division
    d = imul(c, INVERSE)
    return d

In [0]:
def mul(x, y):
    # local computation
    z0 = (x[0]*y[0] + x[0]*y[1] + x[1]*y[0]) % Q
    z1 = (x[1]*y[1] + x[1]*y[2] + x[2]*y[1]) % Q
    z2 = (x[2]*y[2] + x[2]*y[0] + x[0]*y[2]) % Q
    # reshare and distribute
    Z = [ share(z0), share(z1), share(z2) ]
    w = [ sum(row) % Q for row in zip(*Z) ]
    # bring precision back down from double to single
    v = truncate(w)
    return v

In [0]:
class SecureRational(object):
    
    def secure(secret):
        z = SecureRational()
        z.shares = share(encode(secret))
        return z
    
    def reveal(self):
        return decode(reconstruct(reshare(self.shares)))
    
    def __repr__(self):
        return "SecureRational(%f)" % self.reveal()
    
    def __add__(x, y):
        z = SecureRational()
        z.shares = add(x.shares, y.shares)
        return z
    
    def __sub__(x, y):
        z = SecureRational()
        z.shares = sub(x.shares, y.shares)
        return z
    
    def __mul__(x, y):
        z = SecureRational()
        z.shares = mul(x.shares, y.shares)
        return z
    
    def __pow__(x, e):
        z = SecureRational.secure(1)
        for _ in range(e):
            z = z * x
        return z

In [0]:
x = SecureRational.secure(10)
y = SecureRational.secure(33)
z = x * y
assert(z.reveal() == (10) * (33))
print(z.reveal())
print(x.shares)
r = x**2
print(r.reveal())

330.0
[238179356851767236391648175683354116803, 147467239380868698954436621121541361956, 202300094717698558794805758756836010067]
100.0


In [0]:

class OpenRational(object):
    
    def __init__(self, secret):
        self.secret = secret
    
    def secure(secret):
        return OpenRational(secret)
    
    def reveal(self):
        return self.secret
    
    def __repr__(self):
        return "OpenRational(%f)" % self.reveal()
    
    def __add__(x, y):
        return OpenRational(x.secret + y.secret)
    
    def __sub__(x, y):
        return OpenRational(x.secret - y.secret)
    
    def __mul__(x, y):
        return OpenRational(x.secret * y.secret)
    
    def __pow__(x, e):
        z = OpenRational(1)
        for _ in range(e):
            z = z * x
        return z

In [0]:
print(2*np.random.random((3,1))-1)

[[-0.11609057]
 [-0.5895786 ]
 [ 0.30557877]]


In [0]:
# helper functions to map array of numbers to and from secure data type
secure = np.vectorize(lambda x: SecureRational.secure(x))
reveal = np.vectorize(lambda x: x.reveal())

In [0]:

class TwoLayerNetwork:

    def __init__(self, sigmoid):
        self.sigmoid = sigmoid
    
    def train(self, X, y, iterations=1000, alpha=1):

        # prepare alpha value
        alpha = secure(alpha)
        
        # initial weights 
        self.synapse0 = secure(2 * np.random.random((3,1)) - 1)

        # training
        for i in range(iterations):

            # forward propagation
            layer0 = X
            layer1 = self.sigmoid.evaluate(np.dot(layer0, self.synapse0))

            # back propagation
            layer1_error = y - layer1
            layer1_delta = layer1_error * self.sigmoid.derive(layer1)
            
            # provide error feed by revealing values
            if (i+1) % (iterations//10) == 0:
                print("Error: %s" % np.mean(np.abs(reveal(layer1_error))))

            # update
            self.synapse0 += np.dot(layer0.T, layer1_delta) * alpha
            
    def predict(self, X):
        layer0 = X
        layer1 = self.sigmoid.evaluate(np.dot(layer0, self.synapse0))
        return layer1
    
    def print_weights(self):
        print("Layer 0 weights: \n%s" % self.synapse0)

In [0]:
class ThreeLayerNetwork:

    def __init__(self, sigmoid):
        self.sigmoid = sigmoid
    
    def train(self, X, y, iterations=1000, alpha=1):

        # prepare alpha value
        alpha = secure(alpha)
        
        # initial weights
        self.synapse0 = secure(2 * np.random.random((3,4)) - 1)
        self.synapse1 = secure(2 * np.random.random((4,1)) - 1)

        # training
        for i in range(iterations):
    
            # forward propagation
            layer0 = X
            layer1 = self.sigmoid.evaluate(np.dot(layer0, self.synapse0))
            layer2 = self.sigmoid.evaluate(np.dot(layer1, self.synapse1))

            # back propagation
            layer2_error = y - layer2
            layer2_delta = layer2_error * self.sigmoid.derive(layer2)
            layer1_error = np.dot(layer2_delta, self.synapse1.T)
            layer1_delta = layer1_error * self.sigmoid.derive(layer1)

            # provide error feedback by revealing values
            if (i+1) % (iterations//10) == 0:
                print("Error: %s" % np.mean(np.abs(reveal(layer2_error))))

            # update
            self.synapse1 += np.dot(layer1.T, layer2_delta) * alpha
            self.synapse0 += np.dot(layer0.T, layer1_delta) * alpha
            
    def predict(self, X):
        layer0 = X
        layer1 = self.sigmoid.evaluate(np.dot(layer0, self.synapse0))
        layer2 = self.sigmoid.evaluate(np.dot(layer1, self.synapse1))
        return layer2
    
    def print_weights(self):
        print("Layer 0 weights: \n%s" % self.synapse0)
        print("Layer 1 weights: \n%s" % self.synapse1)

In [0]:
class SigmoidMaclaurin3:
    
    def __init__(self):
        ONE = SecureRational.secure(1)
        W0  = SecureRational.secure(1/2)
        W1  = SecureRational.secure(1/4)
        W3  = SecureRational.secure(-1/48)
        self.sigmoid = np.vectorize(lambda x: W0 + (x * W1) + (x**3 * W3))
        self.sigmoid_deriv = np.vectorize(lambda x: (ONE - x) * x)
        
    def evaluate(self, x):
        return self.sigmoid(x)

    def derive(self, x):
        return self.sigmoid_deriv(x)
class SigmoidMaclaurin5:
    
    def __init__(self):
        ONE = SecureRational.secure(1)
        W0  = SecureRational.secure(1/2)
        W1  = SecureRational.secure(1/4)
        W3  = SecureRational.secure(-1/48)
        W5  = SecureRational.secure(1/480)
        self.sigmoid = np.vectorize(lambda x: W0 + (x * W1) + (x**3 * W3) + (x**5 * W5))
        self.sigmoid_deriv = np.vectorize(lambda x:(ONE - x) * x)
        
    def evaluate(self, x):
        return self.sigmoid(x)

    def derive(self, x):
        return self.sigmoid_deriv(x)
    
class SigmoidMaclaurin9:
    
    def __init__(self):
        ONE = SecureRational.secure(1)
        W0  = SecureRational.secure(1/2)
        W1  = SecureRational.secure(1/4)
        W3  = SecureRational.secure(-1/48)
        W5  = SecureRational.secure(1/480)
        W7  = SecureRational.secure(-17/80640)
        W9  = SecureRational.secure(31/1451520)
        self.sigmoid = np.vectorize(lambda x: W0 + (x * W1) + (x**3 * W3) + (x**5 * W5) + (x**7 * W7) + (x**9 * W9))
        self.sigmoid_deriv = np.vectorize(lambda x:(ONE - x) * x)
        
    def evaluate(self, x):
        return self.sigmoid(x)

    def derive(self, x):
        return self.sigmoid_deriv(x)
    
class SigmoidInterpolated10:
    
    def __init__(self):
        ONE = SecureRational.secure(1)
        W0  = SecureRational.secure(0.5)
        W1  = SecureRational.secure(0.2159198015)
        W3  = SecureRational.secure(-0.0082176259)
        W5  = SecureRational.secure(0.0001825597)
        W7  = SecureRational.secure(-0.0000018848)
        W9  = SecureRational.secure(0.0000000072)
        self.sigmoid = np.vectorize(lambda x: W0 + (x * W1) + (x**3 * W3) + (x**5 * W5) + (x**7 * W7) + (x**9 * W9))
        self.sigmoid_deriv = np.vectorize(lambda x:(ONE - x) * x)
        
    def evaluate(self, x):
        return self.sigmoid(x)

    def derive(self, x):
        return self.sigmoid_deriv(x)

In [0]:
X_full = np.array([
            [0,0,0],
            [0,0,1],
            [0,1,0],
            [0,1,1],
            [1,0,0],
            [1,0,1],
            [1,1,0],
            [1,1,1]
        ])

def evaluate(network):
    for x in X_full:
        score = reveal(network.predict(secure(x)))
        prediction = 1 if score > 0.5 else 0
        print("Prediction on %s: %s (%.8f)" % (x, prediction, score))

In [0]:

X_train = np.array([
            [0,0,1],
            [0,1,1],
            [1,0,1],
            [1,1,1]
        ])

y_train = np.array([[
            0,
            0,
            1,
            1
        ]]).T

In [0]:
# reseed to get reproducible results
random.seed(1)
np.random.seed(1)

# pick approximation
sigmoid = SigmoidMaclaurin3()

# train
network = ThreeLayerNetwork(sigmoid)
network.train(secure(X_train), secure(y_train), 500)
network.print_weights()

# evaluate predictions
evaluate(network)

Error: 7.349333460553483e+29
Error: 7.202347253497748e+29
Error: 1.021557364846618e+30
Error: 5.952960445733715e+29
Error: 1.0068586958445364e+30
Error: 6.54090704360681e+29
Error: 8.231253571281738e+29
Error: 5.512000283875262e+29
Error: 7.496320536359282e+29
Error: 4.630080158561821e+29
Layer 0 weights: 
[[SecureRational(-676131635355601824980815314944.000000)
  SecureRational(117596567957933067653374541824.000000)
  SecureRational(29404685140511850577453383680.000000)
  SecureRational(-1117091091202080975386119766016.000000)]
 [SecureRational(970119410201829529631886147584.000000)
  SecureRational(-999502149554020604187024293888.000000)
  SecureRational(852529879103373677352198340608.000000)
  SecureRational(146994337715222683376892772352.000000)]
 [SecureRational(676146112044109373040362520576.000000)
  SecureRational(-911310174102698928134550454272.000000)
  SecureRational(235185850609941379862430220288.000000)
  SecureRational(1352284667518503827737417875456.000000)]]
Layer 1 wei

In [0]:
secure(2 * np.random.random((3,1)) - 1)

array([[SecureRational(-0.165390)],
       [SecureRational(0.117380)],
       [SecureRational(-0.719226)]], dtype=object)

In [0]:
Q = 8191
def encode(x):
    return x % Q

def decode(x):
    return x if x <= Q/2 else x-Q

In [0]:
# using Horner's rule

def evaluate_at_point(coefs, point):
    result = 0
    for coef in reversed(coefs):
        result = (coef + point * result) % Q
    return result

def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

# from http://www.ucl.ac.uk/~ucahcjm/combopt/ext_gcd_python_programs.pdf
def egcd_binary(a,b):
    u, v, s, t, r = 1, 0, 0, 1, 0
    while (a % 2 == 0) and (b % 2 == 0):
        a, b, r = a//2, b//2, r+1
    alpha, beta = a, b
    while (a % 2 == 0):
        a = a//2
        if (u % 2 == 0) and (v % 2 == 0):
            u, v = u//2, v//2
        else:
            u, v = (u + beta)//2, (v - alpha)//2
    while a != b:
        if (b % 2 == 0):
            b = b//2
            if (s % 2 == 0) and (t % 2 == 0):
                s, t = s//2, t//2
            else:
                s, t = (s + beta)//2, (t - alpha)//2
        elif b < a:
            a, b, u, v, s, t = b, a, s, t, u, v
        else:
            b, s, t = b - a, s - u, t - v
    return (2 ** r) * a, s, t


def inverse(a):
    _, b, _ = egcd_binary(a, Q)
    return b

# see https://en.wikipedia.org/wiki/Lagrange_polynomial

def lagrange_constants_for_point(points, point):
    constants = [0] * len(points)
    for i in range(len(points)):
        xi = points[i]
        num = 1
        denum = 1
        for j in range(len(points)):
            if j != i:
                xj = points[j]
                num = (num * (xj - point)) % Q
                denum = (denum * (xj - xi)) % Q
        constants[i] = (num * inverse(denum)) % Q
    return constants

def interpolate_at_point(points_values, point):
    points, values = zip(*points_values)
    constants = lagrange_constants_for_point(points, point)
    return sum( vi * ci for vi, ci in zip(values, constants) ) % Q



In [0]:
#shamir sharing
N = 10
T = 4

assert(T+1 <= N)

def sample_shamir_polynomial(zero_value):
    coefs = [zero_value] + [random.randrange(Q) for _ in range(T)]
    return coefs

SHARE_POINTS = [ p for p in range(1, N+1) ]
assert(0 not in SHARE_POINTS)

def shamir_share(secret):
    polynomial = sample_shamir_polynomial(secret)
    shares = [ evaluate_at_point(polynomial, p) for p in SHARE_POINTS ]
    return shares

def shamir_reconstruct(shares):
    polynomial = [ (p,v) for p,v in zip(SHARE_POINTS, shares) if v is not None ]
    secret = interpolate_at_point(polynomial, 0)
    return secret

shares = shamir_share(5)
for i in range(N-(T+1)):
    shares[i] = None
#shares[-1] = None  # would fail; we need T+K points to reconstruct
x = shamir_reconstruct(shares)
assert(x == 5)

def shamir_add(x, y):
    return [ (xi + yi) % Q for xi, yi in zip(x, y) ]

def shamir_sub(x, y):
    return [ (xi - yi) % Q for xi, yi in zip(x, y) ]


def shamir_mul(x, y):
    return [ (xi * yi) % Q for xi, yi in zip(x, y) ]


In [0]:
class Shamir:
    
    def __init__(self, secret=None):
        self.shares = shamir_share(encode(secret)) if secret is not None else []
        self.degree = T
    
    def reveal(self):
        #assert(self.degree+1 <= N)
        return decode(shamir_reconstruct(self.shares))
    
    def __repr__(self):
        return "Shamir(%d)" % self.reveal()
    
    def __add__(x, y):
        z = Shamir()
        z.shares = shamir_add(x.shares, y.shares)
        z.degree = max(x.degree, y.degree)
        return z
    
    def __sub__(x, y):
        z = Shamir()
        z.shares = shamir_sub(x.shares, y.shares)
        z.degree = max(x.degree, y.degree)
        return z
    
    def __mul__(x, y):
        z = Shamir()
        z.shares = shamir_mul(x.shares, y.shares)
        z.degree = x.degree + y.degree
        return z
    
    def __pow__(x, e):
        #z = Shamir(1)
        z =  Shamir(1);
        for _ in range(e):
            z = z * x   #Shamir(1) * x
            a = z.reveal()
            z = Shamir(a)
        return z
            
       
    
x = Shamir(2)
print(x)

y = Shamir(3)
print(y)

z = x - y
print(z)
assert(z.reveal() == -1)

v = x * y
print(v)
assert(v.reveal() == 6)

po = x**6
print(po)
po.reveal()

Shamir(2)
Shamir(3)
Shamir(-1)
Shamir(6)
Shamir(64)


64

In [0]:
a = Shamir(1)* Shamir(2)
a.reveal()

2

In [0]:
shamir_share(encode(2))

[5569, 7040, 3974, 4414, 314, 303, 5112, 7574, 815, 2827]

In [0]:
# helper functions to map array of numbers to and from secure data type
secure = np.vectorize(lambda x: Shamir(x))
reveal = np.vectorize(lambda x: x.reveal())

In [0]:
shamir_share(encode(3))

[5528, 6351, 6061, 6128, 5903, 2618, 7768, 6156, 5039, 1364]

In [0]:
class TwoLayerNetwork:

    def __init__(self, sigmoid):
        self.sigmoid = sigmoid
    
    def train(self, X, y, iterations=1000, alpha=1):

        # prepare alpha value
        alpha = secure(alpha)
        
        # initial weights 
        self.synapse0 = secure(2 * np.random.random((3,1)) - 1)

        # training
        for i in range(iterations):

            # forward propagation
            layer0 = X
            layer1 = self.sigmoid.evaluate(np.dot(layer0, self.synapse0))

            # back propagation
            layer1_error = y - layer1
            layer1_delta = layer1_error * self.sigmoid.derive(layer1)
            
            # provide error feed by revealing values
            if (i+1) % (iterations//10) == 0:
                print("Error: %s" % np.mean(np.abs(reveal(layer1_error))))

            # update
            self.synapse0 += np.dot(layer0.T, layer1_delta) * alpha
            
    def predict(self, X):
        layer0 = X
        layer1 = self.sigmoid.evaluate(np.dot(layer0, self.synapse0))
        return layer1
    
    def print_weights(self):
        print("Layer 0 weights: \n%s" % self.synapse0)

In [0]:
class ThreeLayerNetwork:

    def __init__(self, sigmoid):
        self.sigmoid = sigmoid
    
    def train(self, X, y, iterations=1000, alpha=1):

        # prepare alpha value
        alpha = secure(alpha)
        
        # initial weights
        self.synapse0 = secure(2 * np.random.random((3,4)) - 1)
        self.synapse1 = secure(2 * np.random.random((4,1)) - 1)

        # training
        for i in range(iterations):
    
            # forward propagation
            layer0 = X
            layer1 = self.sigmoid.evaluate(np.dot(layer0, self.synapse0))
            layer2 = self.sigmoid.evaluate(np.dot(layer1, self.synapse1))

            # back propagation
            layer2_error = y - layer2
            layer2_delta = layer2_error * self.sigmoid.derive(layer2)
            layer1_error = np.dot(layer2_delta, self.synapse1.T)
            layer1_delta = layer1_error * self.sigmoid.derive(layer1)

            # provide error feedback by revealing values
            if (i+1) % (iterations//10) == 0:
                print("Error: %s" % np.mean(np.abs(reveal(layer2_error))))

            # update
            self.synapse1 += np.dot(layer1.T, layer2_delta) * alpha
            self.synapse0 += np.dot(layer0.T, layer1_delta) * alpha
            
    def predict(self, X):
        layer0 = X
        layer1 = self.sigmoid.evaluate(np.dot(layer0, self.synapse0))
        layer2 = self.sigmoid.evaluate(np.dot(layer1, self.synapse1))
        return layer2
    
    def print_weights(self):
        print("Layer 0 weights: \n%s" % self.synapse0)
        print("Layer 1 weights: \n%s" % self.synapse1)

In [0]:
class Sigmoid_Shamir3:
    
    def __init__(self):
        ONE = Shamir(1)
        W0  = Shamir(1/2)
        W1  = Shamir(1/4)
        W3  = Shamir(-1/48)
        self.sigmoid = np.vectorize(lambda x: W0 + (x * W1) + (x**3 * W3))
        self.sigmoid_deriv = np.vectorize(lambda x: (ONE - x) * x)
        
    def evaluate(self, x):
        return self.sigmoid(x)

    def derive(self, x):
        return self.sigmoid_deriv(x)

In [0]:
class Sigmoid_Shamir9:
    
    def __init__(self):
        ONE = Shamir(1)
        W0  = Shamir(1/2)
        W1  = Shamir(1/4)
        W3  = Shamir(-1/48)
        W5  = Shamir(1/480)
        W7  = Shamir(-17/80640)
        W9  = Shamir(31/1451520)
        self.sigmoid = np.vectorize(lambda x: W0 + (x * W1) + (x**3 * W3) + (x**5 * W5) + (x**7 * W7) + (x**9 * W9))
        self.sigmoid_deriv = np.vectorize(lambda x:(ONE - x) * x)
        
    def evaluate(self, x):
        return self.sigmoid(x)

    def derive(self, x):
        return self.sigmoid_deriv(x)

In [0]:
class SigmoidInterpolated10_Shamir:
    
    def __init__(self):
        ONE = Shamir(1)
        W0  = Shamir(0.5)
        W1  = Shamir(0.2159198015)
        W3  = Shamir(-0.0082176259)
        W5  = Shamir(0.0001825597)
        W7  = Shamir(-0.0000018848)
        W9  = Shamir(0.0000000072)
        self.sigmoid = np.vectorize(lambda x: W0 + (x * W1) + (x**3 * W3) + (x**5 * W5) + (x**7 * W7) + (x**9 * W9))
        self.sigmoid_deriv = np.vectorize(lambda x:(ONE - x) * x)
        
    def evaluate(self, x):
        return self.sigmoid(x)

    def derive(self, x):
        return self.sigmoid_deriv(x)

In [0]:
X_full = np.array([
            [0,0,0],
            [0,0,1],
            [0,1,0],
            [0,1,1],
            [1,0,0],
            [1,0,1],
            [1,1,0],
            [1,1,1]
        ])

def evaluate(network):
    for x in X_full:
        score = reveal(network.predict(secure(x)))
        prediction = 1 if score > 0.5 else 0
        print("Prediction on %s: %s (%.8f)" % (x, prediction, score))


X_train = np.array([
            [0,0,1],
            [0,1,1],
            [1,0,1],
            [1,1,1]
        ])

y_train = np.array([[
            0,
            1,
            1,
            0
        ]]).T

In [0]:
# reseed to get reproducible results
random.seed(1)
np.random.seed(1)

# pick approximation
sigmoid = Sigmoid_Shamir9()

# train
network = ThreeLayerNetwork(sigmoid)
network.train(secure(X_train), secure(y_train), 10000)
network.print_weights()

# evaluate predictions
evaluate(network)

Error: 2543.0523524805903
Error: 3114.206687889993
Error: 1707.3038784861565


KeyboardInterrupt: 